In [60]:
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

import json
import praw
import requests
import subprocess

from praw.models import InlineGif, InlineImage, InlineVideo

In [38]:
result = pd.read_pickle('./2_post_content.pkl')

In [69]:
c1 = (~result['author'].isna())
c2 = (~result['style'].isna())
c3 = (result['comment_count'] == 0)
c4 = np.array([int(x[1:-1].split('x')[0]) > 6000 for x in result.original_size.values])
c5 = (~result['eobrowser_link'].isna())
c6 = np.array([len(x) > 0 for x in result.dates])

cuts = c1 & c2 & c3 & c4 & c5 & c6
selection = result[cuts].sort_values('view_count', ascending=False).to_dict(orient='records')

In [16]:
subr = 'EarthFromAbove'
credentials = 'client_secrets.json'
 
with open(credentials) as f:
    creds = json.load(f)

In [17]:
reddit = praw.Reddit(client_id=creds['client_id'],
                     client_secret=creds['client_secret'],
                     user_agent=creds['user_agent'],
                     redirect_uri=creds['redirect_uri'],
                     refresh_token=creds['refresh_token'])
 
reddit.validate_on_submit = True
subreddit = reddit.subreddit(subr) 

In [75]:
def reddit_post(sel):
    title = f'{sel["title"]} {sel["image_size"]} [OC]'

    description = f'Location: {sel["title"]}\n\n' \
    f'{"Date: " + sel["dates"][0] if len(sel["dates"]) == 1 else "Time-range: " + " - ".join(sel["dates"])}\n\n' \
    f'Data: {sel["style"]}\n\n' \
    'Contains modified Copernicus Sentinel data, processed by [Sentinel Hub](https://www.sentinel-hub.com/)\n\n' \
    f'[Link to original resolution {sel["original_size"]}]({sel["page_url"]}/sizes/)\n\n' \
    f'[Inspect in EO Browser]({sel["eobrowser_link"]})\n\n'

    subprocess.call(f'wget {sel["dl_link"]} -O ./images/{sel["entry_id"]}.png', shell=True)

    submission = subreddit.submit_image(title, image_path=f'/home/mlubej/play/pet_projects/reddit_post/images/{sel["entry_id"]}.png')
    comment = submission.reply(description)

In [79]:
for i in range(11, 21):
    try:
        reddit_post(selection[i])
    except:
        print('failed for', i)